In [ ]:
!pip uninstall fastai torch torchaudio fastcore torchvision -y 

In [ ]:
!pip install ../input/packages/packages/packages/colorednoise-1.1.1/colorednoise-1.1.1 --find-links ../input/packages/packages --no-index --use-feature=2020-resolver

In [ ]:
!pip install ../input/packages/fastaudio-0.0.post0.dev143gc7a2b85.dirty-py2.py3-none-any.whl --find-links ../input/packages/packages/packages --no-index --verbose --upgrade --use-feature=2020-resolver

In [ ]:
import fastaudio

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

# Map 1 library
import plotly.express as px

# Map 2 libraries
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Librosa Libraries
import librosa
import librosa.display
import IPython.display as ipd

import sklearn

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import soundfile as sf

### resample all audio files to 32 khz

In [ ]:
#import ffmpy

In [ ]:
#new_path = Path("data/train_audio_resample")
#old_path = Path("data/train_audio")

In [ ]:
#new_path.mkdir()

In [ ]:
#for subfolder in old_path.ls():
#    Path(str(subfolder).replace("/train_audio/", "/train_audio_resample/")).mkdir()

In [ ]:
#audio_files = get_files(old_path)

In [ ]:
#from fastprogress import progress_bar

In [ ]:
#for file in progress_bar(audio_files):
#    new_file = Path(str(file).replace("/train_audio/", "/train_audio_resample/").replace(".mp3", ".wav"))
#    new_file.parent.mkdir(exist_ok=True)
#    ff = ffmpy.FFmpeg(inputs={str(file):None}, outputs={str(new_file): "-ar 32000 -ac 1"})
#    ff.run()

In [ ]:
#resampled_audio_files = get_files(new_path)

In [ ]:
#len(resampled_audio_files)

In [ ]:
#len(audio_files)

In [ ]:
# Import data
train_csv = pd.read_csv("../input/birdsong-recognition/train.csv")
test_csv = pd.read_csv("../input/birdsong-recognition/test.csv")

# Create some time features
train_csv['year'] = train_csv['date'].apply(lambda x: x.split('-')[0])
train_csv['month'] = train_csv['date'].apply(lambda x: x.split('-')[1])
train_csv['day_of_month'] = train_csv['date'].apply(lambda x: x.split('-')[2])

print("There are {:,} unique bird species in the dataset.".format(len(train_csv['species'].unique())))

In [ ]:
train_csv["file_path"] = train_csv[["ebird_code", "filename"]].agg("/".join, axis=1)
train_csv["file_path"] = train_csv["file_path"]#.apply(lambda x: x.split(".")[0] + ".wav")

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

In [ ]:
cfg = AudioConfig.BasicMelSpectrogram(n_fft=512)
a2s = AudioToSpec.from_cfg(cfg)

In [ ]:
path = Path("../input/birdsong-recognition")

In [ ]:
auds = DataBlock(blocks=(partial(AudioBlock, crop_signal_to=5000), MultiCategoryBlock),  
                 get_x=ColReader("file_path", pref=path/"train_audio"), 
                 batch_tfms = [a2s],
                 get_y=lambda x: [ColReader("ebird_code")(x)])

In [ ]:
dbunch = auds.dataloaders(train_csv, bs=64)

In [ ]:
x,y = dbunch.one_batch()

In [ ]:
x.shape

In [ ]:
dbunch.show_batch(figsize=(10, 5))

In [ ]:
learn = cnn_learner(dbunch, 
            xresnet50, 
            config=cnn_config(n_in=1), #<- Only audio specific modification here
            pretrained=True)

In [ ]:
#learn.lr_find()

In [ ]:
from fastai.callback.training import ShortEpochCallback

In [ ]:
learn.fit_one_cycle(1, slice(1e-2), cbs=[ShortEpochCallback()])

In [ ]:
#learn.save("xresnet18-10epoch") 

# inference code

## split files

In [ ]:
test_csv = pd.read_csv("../input/birdcall-check/test.csv")

In [ ]:
test_csv.head()

In [ ]:
out_dir = Path("test_audio_splitted")
out_dir.mkdir()

In [ ]:
row_ids_splitted = []
filenames_splitted = []

for audio_id in progress_bar(list(test_csv["audio_id"].unique())):

    audio, orig_sr = librosa.load(f"../input/birdcall-check/test_audio/{audio_id}.mp3")
    sr = 32_000
    audio = librosa.resample(audio, orig_sr, sr)
    duration = len(audio) / sr

    segment_df = test_csv[test_csv["audio_id"] == audio_id]
    site = segment_df["site"].iloc[0]
    
    
    if (site == "site_1") or (site == "site_2"):
        for row_id, end_second in zip(segment_df["row_id"], segment_df["seconds"]):
            end_idx = math.floor(end_second * sr)
            start_idx = math.floor(end_idx - (5 * sr))
            audio_slice = audio[start_idx:end_idx]
            filename = f"{row_id}.wav"
            sf.write(out_dir/f"{filename}", audio_slice, sr)
            row_ids_splitted.append(row_id)
            filenames_splitted.append(filename)

    if site == "site_3":
        row_id = segment_df["row_id"].iloc[0]
        end_second = 5
        while end_second < duration:
            end_idx = math.floor(end_second * sr)
            start_idx = math.floor(end_idx - (5 * sr))
            audio_slice = audio[start_idx:end_idx]
            filename = f"{row_id}_{end_second}.wav"
            sf.write(out_dir/f"{filename}", audio_slice, sr)
            row_ids_splitted.append(row_id)
            filenames_splitted.append(filename)
            end_second += 5

test_df_splitted = pd.DataFrame(data = {"row_id": row_ids_splitted, "filename": filenames_splitted})

In [ ]:
test_df_splitted

In [ ]:
test_datablock = DataBlock(blocks=(partial(AudioBlock, crop_signal_to=5000), MultiCategoryBlock),  
                     get_x=ColReader("filename", pref=out_dir), 
                     batch_tfms = [a2s],
                     get_y=lambda x: [ColReader("row_id")(x)])

In [ ]:
fake_dls = test_datablock.dataloaders(test_df_splitted)

In [ ]:
test_dl = fake_dls.test_dl(test_df_splitted)

In [ ]:
sd = torch.load("../input/birdstrainedmodels/xresnet50-unfreeze-5.pth")

In [ ]:
learn.model.load_state_dict(sd["model"])

In [ ]:
preds = learn.get_preds(dl = test_dl, )

In [ ]:
len(test_df_splitted)

In [ ]:
x= next(iter(test_dl))

In [ ]:
x[0]

In [ ]:
pred = learn.model(x[0])

In [ ]:
F.sigmoid(pred).max()